In [1]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from funciones import normalizar_nombre

import time

In [16]:
def get_urls(check_in, check_out):
    # Formato check_in y check_out --> y%-%m-%d --> 2024-09-06
    hoteles_sitios = {
        'Andalucía' : f'https://www.booking.com/searchresults.es.html?ss=Andaluc%C3%ADa%2C+Espa%C3%B1a&ssne=Ceuta&ssne_untouched=Ceuta&highlighted_hotels=2542021&efdco=1&label=madrid-qZauT3iEXUUmv8HKzOiiqgS638127213283%3Apl%3Ata%3Ap11660%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-1586918843%3Alp9198434%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6olzyFZA51Co&sid=d8f29e7bf8218ffd361728fa13b9e30b&aid=303946&lang=es&sb=1&src_elem=sb&src=searchresults&dest_id=725&dest_type=region&ac_position=0&ac_click_type=b&ac_langcode=es&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=6f0a44b7bc1108f1&ac_meta=GhA2ZjBhNDRiN2JjMTEwOGYxIAAoATICZXM6CUFuZGFsdWNpYUAASgBQAA%3D%3D&checkin={check_in}&checkout={check_out}&group_adults=2&no_rooms=1&group_children=0',
        'Aragón' : f'https://www.booking.com/searchresults.es.html?ss=Arag%C3%B3n%2C+Espa%C3%B1a&ssne=Andaluc%C3%ADa&ssne_untouched=Andaluc%C3%ADa&highlighted_hotels=2542021&label=madrid-qZauT3iEXUUmv8HKzOiiqgS638127213283%3Apl%3Ata%3Ap11660%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-1586918843%3Alp9198434%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6olzyFZA51Co&sid=d8f29e7bf8218ffd361728fa13b9e30b&aid=303946&lang=es&sb=1&src_elem=sb&src=searchresults&dest_id=726&dest_type=region&ac_position=1&ac_click_type=b&ac_langcode=es&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=830544dff2fd02b6&ac_meta=GhA4MzA1NDRkZmYyZmQwMmI2IAEoATICZXM6BEFyYWdAAEoAUAA%3D&checkin={check_in}&checkout={check_out}&group_adults=2&no_rooms=1&group_children=0',
        'Asturias' : f'https://www.booking.com/searchresults.es.html?ss=Asturias%2C+Espa%C3%B1a&ssne=Arag%C3%B3n&ssne_untouched=Arag%C3%B3n&highlighted_hotels=2542021&label=madrid-qZauT3iEXUUmv8HKzOiiqgS638127213283%3Apl%3Ata%3Ap11660%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-1586918843%3Alp9198434%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6olzyFZA51Co&sid=d8f29e7bf8218ffd361728fa13b9e30b&aid=303946&lang=es&sb=1&src_elem=sb&src=searchresults&dest_id=727&dest_type=region&ac_position=0&ac_click_type=b&ac_langcode=es&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=28e4450c8b010454&ac_meta=GhAyOGU0NDUwYzhiMDEwNDU0IAAoATICZXM6CEFzdHVyaWFzQABKAFAA&checkin={check_in}&checkout={check_out}&group_adults=2&no_rooms=1&group_children=0',
        'Islas Baleares' : f'https://www.booking.com/searchresults.es.html?ss=Islas+Baleares%2C+Espa%C3%B1a&ssne=Asturias&ssne_untouched=Asturias&highlighted_hotels=2542021&label=madrid-qZauT3iEXUUmv8HKzOiiqgS638127213283%3Apl%3Ata%3Ap11660%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-1586918843%3Alp9198434%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6olzyFZA51Co&sid=d8f29e7bf8218ffd361728fa13b9e30b&aid=303946&lang=es&sb=1&src_elem=sb&src=searchresults&dest_id=728&dest_type=region&ac_position=0&ac_click_type=b&ac_langcode=es&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=932d4519602700b6&ac_meta=GhA5MzJkNDUxOTYwMjcwMGI2IAAoATICZXM6CEJhbGVhcmVzQABKAFAA&checkin={check_in}&checkout={check_out}&group_adults=2&no_rooms=1&group_children=0',
        'Islas Canarias' : f'https://www.booking.com/searchresults.es.html?ss=Islas+Canarias%2C+Espa%C3%B1a&ssne=Islas+Baleares&ssne_untouched=Islas+Baleares&highlighted_hotels=2542021&label=madrid-qZauT3iEXUUmv8HKzOiiqgS638127213283%3Apl%3Ata%3Ap11660%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-1586918843%3Alp9198434%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6olzyFZA51Co&sid=d8f29e7bf8218ffd361728fa13b9e30b&aid=303946&lang=es&sb=1&src_elem=sb&src=searchresults&dest_id=730&dest_type=region&ac_position=0&ac_click_type=b&ac_langcode=es&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=95934524bf550305&ac_meta=GhA5NTkzNDUyNGJmNTUwMzA1IAAoATICZXM6DklzbGFzIENhbmFyaWFzQABKAFAA&checkin={check_in}&checkout={check_out}&group_adults=2&no_rooms=1&group_children=0',
        'Cantabria' : f'https://www.booking.com/searchresults.es.html?ss=Cantabria&ssne=Cantabria&ssne_untouched=Cantabria&highlighted_hotels=2542021&label=madrid-qZauT3iEXUUmv8HKzOiiqgS638127213283%3Apl%3Ata%3Ap11660%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-1586918843%3Alp9198434%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6olzyFZA51Co&sid=d8f29e7bf8218ffd361728fa13b9e30b&aid=303946&lang=es&sb=1&src_elem=sb&src=searchresults&dest_id=731&dest_type=region&checkin={check_in}&checkout={check_out}&group_adults=2&no_rooms=1&group_children=0',
        'Castilla y León' : f'https://www.booking.com/searchresults.es.html?ss=Castilla+y+Le%C3%B3n&ssne=Castilla+y+Le%C3%B3n&ssne_untouched=Castilla+y+Le%C3%B3n&highlighted_hotels=2542021&label=madrid-qZauT3iEXUUmv8HKzOiiqgS638127213283%3Apl%3Ata%3Ap11660%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-1586918843%3Alp9198434%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6olzyFZA51Co&sid=d8f29e7bf8218ffd361728fa13b9e30b&aid=303946&lang=es&sb=1&src_elem=sb&src=searchresults&dest_id=733&dest_type=region&checkin={check_in}&checkout={check_out}&group_adults=2&no_rooms=1&group_children=0',
        'Castilla La Mancha' : f'https://www.booking.com/searchresults.es.html?ss=Castilla+La+Mancha&ssne=Castilla+La+Mancha&ssne_untouched=Castilla+La+Mancha&highlighted_hotels=2542021&label=madrid-qZauT3iEXUUmv8HKzOiiqgS638127213283%3Apl%3Ata%3Ap11660%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-1586918843%3Alp9198434%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6olzyFZA51Co&sid=d8f29e7bf8218ffd361728fa13b9e30b&aid=303946&lang=es&sb=1&src_elem=sb&src=searchresults&dest_id=732&dest_type=region&checkin={check_in}&checkout={check_out}&group_adults=2&no_rooms=1&group_children=0',
        'Cataluña' : f'https://www.booking.com/searchresults.es.html?ss=Catalu%C3%B1a&ssne=Catalu%C3%B1a&ssne_untouched=Catalu%C3%B1a&highlighted_hotels=2542021&label=madrid-qZauT3iEXUUmv8HKzOiiqgS638127213283%3Apl%3Ata%3Ap11660%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-1586918843%3Alp9198434%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6olzyFZA51Co&sid=d8f29e7bf8218ffd361728fa13b9e30b&aid=303946&lang=es&sb=1&src_elem=sb&src=searchresults&dest_id=734&dest_type=region&checkin={check_in}&checkout={check_out}&group_adults=2&no_rooms=1&group_children=0',
        'Comunidad Valenciana' : f'https://www.booking.com/searchresults.es.html?ss=Comunidad+Valenciana&ssne=Comunidad+Valenciana&ssne_untouched=Comunidad+Valenciana&highlighted_hotels=2542021&label=madrid-qZauT3iEXUUmv8HKzOiiqgS638127213283%3Apl%3Ata%3Ap11660%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-1586918843%3Alp9198434%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6olzyFZA51Co&sid=d8f29e7bf8218ffd361728fa13b9e30b&aid=303946&lang=es&sb=1&src_elem=sb&src=searchresults&dest_id=1523&dest_type=region&checkin={check_in}&checkout={check_out}&group_adults=2&no_rooms=1&group_children=0',
        'Extremadura' : f'https://www.booking.com/searchresults.es.html?ss=Extremadura&ssne=Extremadura&ssne_untouched=Extremadura&highlighted_hotels=2542021&label=madrid-qZauT3iEXUUmv8HKzOiiqgS638127213283%3Apl%3Ata%3Ap11660%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-1586918843%3Alp9198434%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6olzyFZA51Co&sid=d8f29e7bf8218ffd361728fa13b9e30b&aid=303946&lang=es&sb=1&src_elem=sb&src=searchresults&dest_id=1522&dest_type=region&checkin={check_in}&checkout={check_out}&group_adults=2&no_rooms=1&group_children=0',
        'Galicia' : f'https://www.booking.com/searchresults.es.html?ss=Galicia&ssne=Galicia&ssne_untouched=Galicia&highlighted_hotels=2542021&label=madrid-qZauT3iEXUUmv8HKzOiiqgS638127213283%3Apl%3Ata%3Ap11660%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-1586918843%3Alp9198434%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6olzyFZA51Co&sid=d8f29e7bf8218ffd361728fa13b9e30b&aid=303946&lang=es&sb=1&src_elem=sb&src=searchresults&dest_id=735&dest_type=region&checkin={check_in}&checkout={check_out}&group_adults=2&no_rooms=1&group_children=0',
        'Comunidad de Madrid' : f'https://www.booking.com/searchresults.es.html?ss=Comunidad+de+Madrid&ssne=Comunidad+de+Madrid&ssne_untouched=Comunidad+de+Madrid&highlighted_hotels=2542021&label=madrid-qZauT3iEXUUmv8HKzOiiqgS638127213283%3Apl%3Ata%3Ap11660%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-1586918843%3Alp9198434%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6olzyFZA51Co&sid=d8f29e7bf8218ffd361728fa13b9e30b&aid=303946&lang=es&sb=1&src_elem=sb&src=searchresults&dest_id=765&dest_type=region&checkin={check_in}&checkout={check_out}&group_adults=2&no_rooms=1&group_children=0',
        'Región de Murcia' : f'https://www.booking.com/searchresults.es.html?ss=Murcia&ssne=Murcia&ssne_untouched=Murcia&highlighted_hotels=2542021&label=madrid-qZauT3iEXUUmv8HKzOiiqgS638127213283%3Apl%3Ata%3Ap11660%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-1586918843%3Alp9198434%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6olzyFZA51Co&sid=d8f29e7bf8218ffd361728fa13b9e30b&aid=303946&lang=es&sb=1&src_elem=sb&src=searchresults&dest_id=768&dest_type=region&checkin={check_in}&checkout={check_out}&group_adults=2&no_rooms=2&group_children=0',
        'Comunidad Foral de Navarra' : f'https://www.booking.com/searchresults.es.html?ss=Navarra&ssne=Navarra&ssne_untouched=Navarra&highlighted_hotels=2542021&label=madrid-qZauT3iEXUUmv8HKzOiiqgS638127213283%3Apl%3Ata%3Ap11660%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-1586918843%3Alp9198434%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6olzyFZA51Co&sid=d8f29e7bf8218ffd361728fa13b9e30b&aid=303946&lang=es&sb=1&src_elem=sb&src=searchresults&dest_id=769&dest_type=region&checkin={check_in}&checkout={check_out}&group_adults=2&no_rooms=1&group_children=0',
        'País Vasco' : f'https://www.booking.com/searchresults.es.html?ss=Pa%C3%ADs+Vasco&ssne=Pa%C3%ADs+Vasco&ssne_untouched=Pa%C3%ADs+Vasco&highlighted_hotels=2542021&label=madrid-qZauT3iEXUUmv8HKzOiiqgS638127213283%3Apl%3Ata%3Ap11660%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-1586918843%3Alp9198434%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6olzyFZA51Co&sid=d8f29e7bf8218ffd361728fa13b9e30b&aid=303946&lang=es&sb=1&src_elem=sb&src=searchresults&dest_id=729&dest_type=region&checkin={check_in}&checkout={check_out}&group_adults=2&no_rooms=1&group_children=0',
        'La Rioja' : f'https://www.booking.com/searchresults.es.html?ss=La+Rioja&ssne=La+Rioja&ssne_untouched=La+Rioja&highlighted_hotels=2542021&label=madrid-qZauT3iEXUUmv8HKzOiiqgS638127213283%3Apl%3Ata%3Ap11660%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-1586918843%3Alp9198434%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6olzyFZA51Co&sid=d8f29e7bf8218ffd361728fa13b9e30b&aid=303946&lang=es&sb=1&src_elem=sb&src=searchresults&dest_id=761&dest_type=region&checkin={check_in}&checkout={check_out}&group_adults=2&no_rooms=1&group_children=0',
        #'Ceuta' : f'https://www.booking.com/searchresults.es.html?ss=Ceuta&ssne=Ceuta&ssne_untouched=Ceuta&highlighted_hotels=2542021&label=madrid-qZauT3iEXUUmv8HKzOiiqgS638127213283%3Apl%3Ata%3Ap11660%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-1586918843%3Alp9198434%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6olzyFZA51Co&sid=d8f29e7bf8218ffd361728fa13b9e30b&aid=303946&lang=es&sb=1&src_elem=sb&src=searchresults&dest_id=13158&dest_type=region&checkin={check_in}&checkout={check_out}&group_adults=2&no_rooms=1&group_children=0',
        #'Melilla' : f'https://www.booking.com/searchresults.es.html?ss=Melilla&ssne=Melilla&ssne_untouched=Melilla&highlighted_hotels=2542021&label=madrid-qZauT3iEXUUmv8HKzOiiqgS638127213283%3Apl%3Ata%3Ap11660%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-1586918843%3Alp9198434%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6olzyFZA51Co&sid=d8f29e7bf8218ffd361728fa13b9e30b&aid=303946&lang=es&sb=1&src_elem=sb&src=searchresults&dest_id=-391809&dest_type=city&checkin={check_in}&checkout={check_out}&group_adults=2&no_rooms=1&group_children=0'
    }
    return hoteles_sitios
hoteles = get_urls('2024-11-14', '2024-11-16')
hoteles1 = get_urls('2024-11-14', '2024-11-15')
hoteles

{'Andalucía': 'https://www.booking.com/searchresults.es.html?ss=Andaluc%C3%ADa%2C+Espa%C3%B1a&ssne=Ceuta&ssne_untouched=Ceuta&highlighted_hotels=2542021&efdco=1&label=madrid-qZauT3iEXUUmv8HKzOiiqgS638127213283%3Apl%3Ata%3Ap11660%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-1586918843%3Alp9198434%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6olzyFZA51Co&sid=d8f29e7bf8218ffd361728fa13b9e30b&aid=303946&lang=es&sb=1&src_elem=sb&src=searchresults&dest_id=725&dest_type=region&ac_position=0&ac_click_type=b&ac_langcode=es&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=6f0a44b7bc1108f1&ac_meta=GhA2ZjBhNDRiN2JjMTEwOGYxIAAoATICZXM6CUFuZGFsdWNpYUAASgBQAA%3D%3D&checkin=2024-11-14&checkout=2024-11-16&group_adults=2&no_rooms=1&group_children=0',
 'Aragón': 'https://www.booking.com/searchresults.es.html?ss=Arag%C3%B3n%2C+Espa%C3%B1a&ssne=Andaluc%C3%ADa&ssne_untouched=Andaluc%C3%ADa&highlighted_hotels=2542021&label=madrid-qZauT3iEXUUmv8HKzOiiqgS638127213283%3Apl%3Ata%3Ap11660%3Ap2

In [ ]:
for city, url in hoteles.items():
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    response = requests.get(url, headers=headers)
    html = response.content
    soup = BeautifulSoup(html, 'html.parser')
    format_number = soup.find('h1', class_='e037993315 f8da796a11').text.split()[-3]
    print(city,',', format_number.replace('.',''))

In [10]:
#Esto lo usamos para el Scrapping. El URL es el de Transfermrkt, si nos devuelve una respuesta de 200 es porque es apta

url = "https://www.booking.com/searchresults.es.html?ss=Ceuta%2C+Espa%C3%B1a&ssne=Cuenca&ssne_untouched=Cuenca&highlighted_hotels=2542021&label=madrid-qZauT3iEXUUmv8HKzOiiqgS638127213283%3Apl%3Ata%3Ap11660%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-1586918843%3Alp9198434%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9Ye7BFAsTyVd6olzyFZA51Co&sid=d8f29e7bf8218ffd361728fa13b9e30b&aid=303946&lang=es&sb=1&src_elem=sb&src=searchresults&dest_id=13158&dest_type=region&ac_position=0&ac_click_type=b&ac_langcode=es&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=f8d254efef870099&ac_meta=GhBmOGQyNTRlZmVmODcwMDk5IAAoATICZXM6BWNldXRhQABKAFAA&checkin=2024-08-25&checkout=2024-08-26&group_adults=2&no_rooms=1&group_children=0"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

response = requests.get(url, headers=headers)
response

<Response [200]>

In [18]:
def get_data(url):
    # Configura Selenium con el navegador Chrome
    driver = webdriver.Chrome()

    try:
        driver.get(url)
        try:
            # Esperar a que el botón de consentimiento esté presente y hacer clic
            consent_button = driver.find_element(By.ID, "onetrust-accept-btn-handler")
            consent_button.click()
            print("Botón de consentimiento de cookies encontrado y clicado.")

            try:
                # Usar WebDriverWait para asegurar que el botón de cerrar el popup sea visible y clicable
                close_popup_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, "//button[@aria-label='Ignorar información sobre el inicio de sesión.']"))
                )
                close_popup_button.click()
                print("Popup cerrado.")
            
            except (NoSuchElementException, TimeoutException, ElementClickInterceptedException):
                print("No se encontró el botón de cerrar el popup o no es clicable. Intentando con JavaScript.")
                try:
                    # Intentar cerrar el popup con JavaScript si es necesario
                    driver.execute_script("arguments[0].click();", close_popup_button)
                    print("Popup cerrado con JavaScript.")
                except Exception as e:
                    print(f"No se pudo cerrar el popup: {e}")
         
        except NoSuchElementException:
            print("No se encontró el botón de consentimiento de cookies. Puede que ya se haya cerrado o no sea necesario.")

        # Espera unos segundos para asegurarte de que la página ha terminado de cargar
        time.sleep(2)
        last_height = driver.execute_script("return document.body.scrollHeight")
        counter = 0
        while True:
            # Scroll down en la página
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)  # Espera para que se cargue el nuevo contenido
            try:
                print(counter)
                load_more_button = driver.find_element(By.XPATH, "//button[.//span[text()='Cargar más resultados']]")
                load_more_button.click()
                counter += 1
                print("Botón 'Cargar más resultados' encontrado y clicado.")
                time.sleep(5)  # Esperar a que se carguen los resultados adicionales
            except NoSuchElementException:
                print("No se encontró el botón 'Cargar más resultados'. Se ha alcanzado el final de la lista.")
                if counter >= 5:
                    print("Se han cargado todos los resultados.")
                    print('html Cargado')
                    html = driver.page_source
                    return(html) # Salir del bucle si no hay más botones "Cargar más resultados"
            except ElementClickInterceptedException:
                print("El botón 'Cargar más resultados' fue interceptado. Intentando de nuevo.")
                counter += 1
                time.sleep(2)

            # Verificar si la altura de la página ha cambiado
            last_height = driver.execute_script("return document.body.scrollHeight")
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                print('final de pagina')
                # Si la altura no ha cambiado, se ha alcanzado el final de la página
    finally:
        # Cierra el navegador manualmente después de haber obtenido el HTML
        driver.quit()


In [19]:
counter = 0
for x in [hoteles, hoteles1]:
    df_final = pd.DataFrame(columns=['name','url','precio','puntuacion','localización','n_personas','noches','city'])
    def save_data(html_raw, city):
        df = pd.DataFrame(columns=['name','url','precio','puntuacion','localización','n_personas','noches'])
        soup = BeautifulSoup(html_raw, 'html.parser')
        print(f'total hoteles encontrados en {city}: ', len(soup.find_all('div', attrs={'data-testid' : 'property-card'})))
        for row in soup.find_all('div', attrs={'data-testid' : 'property-card'}):
            name = row.find('div', attrs={'data-testid' : 'title'}).text
            url = row.find('a', class_='f0ebe87f68')['href']
            try:
                precio = row.find('span', attrs={'data-testid': 'price-and-discounted-price'}).text[2:]
            except:
                precio = np.nan
            try:
                puntuacion = row.find('div', class_='d0522b0cca fd44f541d8').text.split()[-1]
            except:
                puntuacion = np.nan
            localización = row.find('span', attrs={'data-testid':'address'}).text
            df.loc[len(df.index)] = [name, url, precio, puntuacion, localización, 2, 2]
        df['city'] = normalizar_nombre(city)
        return df
    for key, value in x.items():
        print(df_final.shape)
        df_temp = save_data(get_data(value), key)
        df_final = pd.concat([df_final, df_temp], axis=0, ignore_index=True)
    if counter == 0:
        df_final.to_csv('hoteles_2p_2n.csv', index=False)
        counter += 1
    else:
        df_final.to_csv('hoteles_2p_1n.csv', index=False)

(0, 8)
Botón de consentimiento de cookies encontrado y clicado.
Popup cerrado.
0
No se encontró el botón 'Cargar más resultados'. Se ha alcanzado el final de la lista.
0
Botón 'Cargar más resultados' encontrado y clicado.
final de pagina
1
Botón 'Cargar más resultados' encontrado y clicado.
final de pagina
2
Botón 'Cargar más resultados' encontrado y clicado.
final de pagina
3
Botón 'Cargar más resultados' encontrado y clicado.
final de pagina
4
Botón 'Cargar más resultados' encontrado y clicado.
final de pagina
5
Botón 'Cargar más resultados' encontrado y clicado.
final de pagina
6
Botón 'Cargar más resultados' encontrado y clicado.
final de pagina
7
Botón 'Cargar más resultados' encontrado y clicado.
final de pagina
8
Botón 'Cargar más resultados' encontrado y clicado.
final de pagina
9
Botón 'Cargar más resultados' encontrado y clicado.
final de pagina
10
Botón 'Cargar más resultados' encontrado y clicado.
final de pagina
11
Botón 'Cargar más resultados' encontrado y clicado.
final 